# 税制政策分析の基礎

DSGEモデルを用いた税制政策の経済効果分析を学習します。

In [17]:
# 環境設定
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 共通インフラの読み込み
sys.path.append('notebooks')
from common import (
    setup_notebook_environment,
    load_baseline_model,
    create_research_simulator
)

# 環境初期化
project_root = setup_notebook_environment("02_tax_policy_fundamentals")

🎌 日本語フォント設定: Hiragino Sans (fallback)
📁 プロジェクトルート: /Users/daisukeyoda/Documents/JapanTaxSimulator
📋 02_tax_policy_fundamentals 環境設定完了


In [18]:
# モデルとシミュレータの初期化
model = load_baseline_model()
simulator, status_info = create_research_simulator(model)

print(f"シミュレータ: {status_info['simulator_type']}")
print(f"研究整合性: {status_info['research_compliance']}")

✅ ベースラインモデル読み込み成功
🔄 定常状態を計算中...
✅ 定常状態計算完了
🎯 Setting up Klein linearization (research-grade)
Adjusting system to be square: 29 equations -> 27 equations
Forward-looking equations (6): [3, 5, 6, 8, 14, 21]
System is overdetermined - removing equations while preserving structure...
Identified critical equations: [9, 10]
Keeping 27 equations: [ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 19 20 21 22 23 24 25 26
 27 28 29]
Square system achieved: (27, 27) with rank 1
A matrix is still rank deficient (1/27)
Applying regularization with parameter: 1.81e-10
Regularization did not help - may need model restructuring
Matrix A shape: (27, 27)
Matrix B shape: (27, 27)
A matrix rank: 1
B matrix rank: 3
Number of explosive eigenvalues: 11
Number of forward-looking variables: 5
This may indicate model indeterminacy or non-existence of solution.
⚠️ Klein linearization failed, falling back to simple method
✅ Setting up simple linearization (demo/educational - NOT DSGE-derived)
✅ Simple linearizatio

/Users/daisukeyoda/Documents/JapanTaxSimulator/src/linearization_improved.py:512: RuntimeWarning: divide by zero encountered in divide
  eigenvalues = alpha / beta
/Users/daisukeyoda/Documents/JapanTaxSimulator/src/linearization_improved.py:512: RuntimeWarning: invalid value encountered in divide
  eigenvalues = alpha / beta
/Users/daisukeyoda/Documents/JapanTaxSimulator/src/simulation/enhanced_simulator.py:91: UserWarning: Blanchard-Kahn conditions not satisfied
  warnings.warn("Blanchard-Kahn conditions not satisfied")


In [19]:
# 現在の税制確認
params = model.params
print("=== 現在の税制 ===")
print(f"消費税率: {params.tau_c:.1%}")
print(f"所得税率: {params.tau_l:.1%}")
print(f"法人税率: {params.tau_f:.1%}")

# 定常状態確認
ss = model.steady_state.to_dict()
print(f"\nGDP: {ss['Y']:.3f}")
print(f"消費: {ss['C']:.3f}")
print(f"投資: {ss['I']:.3f}")

=== 現在の税制 ===
消費税率: 10.0%
所得税率: 20.0%
法人税率: 30.0%

GDP: 0.363
消費: 0.266
投資: 0.019


In [20]:
# 消費税改革シミュレーション
from src.tax_simulator import TaxReform

consumption_tax_reform = TaxReform(
    name="消費税2%引き上げ",
    tau_c=0.12,  # 10% → 12%
    implementation='permanent'
)

print(f"改革案: {consumption_tax_reform.name}")
print(f"消費税率: {params.tau_c:.1%} → {consumption_tax_reform.tau_c:.1%}")

# シミュレーション実行
print("\n🔄 シミュレーション実行中...")
consumption_results = simulator.simulate_reform(
    reform=consumption_tax_reform,
    periods=20
)
print("✅ シミュレーション完了")

改革案: 消費税2%引き上げ
消費税率: 10.0% → 12.0%

🔄 シミュレーション実行中...
Simulating 消費税2%引き上げ with enhanced engine...
Using tax-adjusted initial guess for problematic range (magnitude: 0.020)
✅ シミュレーション完了


/Users/daisukeyoda/Documents/JapanTaxSimulator/src/tax_simulator.py:279: ResearchWarning: RESEARCH WARNING - simulate_reform: Uses automatic model selection (simple vs complex) with different economic assumptions. May return results from different underlying models without clear indication. Welfare calculations use simplified approximations.
  return self.simulator.simulate_reform(reform, periods, compute_welfare=True)
/Users/daisukeyoda/Documents/JapanTaxSimulator/src/tax_simulator.py:156: ResearchWarning: RESEARCH WARNING - simulate_reform: Uses automatic model selection (simple vs complex) with different economic assumptions. May return results from different underlying models without clear indication. Welfare calculations use simplified approximations.
  results = self.simulation_engine.simulate_reform(reform, periods)
/Users/daisukeyoda/Documents/JapanTaxSimulator/src/tax_simulator.py:160: ResearchWarning: RESEARCH WARNING - analyze_welfare_impact: Welfare calculations use simplif

In [21]:
# 結果分析
if hasattr(consumption_results, 'steady_state_baseline') and hasattr(consumption_results, 'steady_state_reform'):
    baseline_ss = consumption_results.steady_state_baseline
    reform_ss = consumption_results.steady_state_reform
    
    print("=== 消費税引き上げの長期的影響 ===")
    
    variables = {'Y': 'GDP', 'C': '消費', 'I': '投資', 'L': '労働時間'}
    
    for var, jp_name in variables.items():
        if hasattr(baseline_ss, var) and hasattr(reform_ss, var):
            old_val = getattr(baseline_ss, var)
            new_val = getattr(reform_ss, var)
            if old_val != 0:
                change = (new_val - old_val) / old_val * 100
                direction = "📈" if change > 0 else "📉" if change < 0 else "➡️"
                print(f"{direction} {jp_name}: {change:+.2f}%")
    
    # 福利厚生への影響
    if hasattr(consumption_results, 'welfare_change'):
        welfare = consumption_results.welfare_change
        welfare_direction = "📈" if welfare > 0 else "📉" if welfare < 0 else "➡️"
        print(f"\n{welfare_direction} 家計福利: {welfare:+.2f}%")
else:
    print("⚠️ 詳細な比較データが利用できません")

=== 消費税引き上げの長期的影響 ===
📉 GDP: -6.56%
📉 消費: -0.90%
➡️ 投資: +0.00%
➡️ 労働時間: +0.00%

📉 家計福利: -0.00%


In [22]:
# 所得税改革シミュレーション
income_tax_reform = TaxReform(
    name="所得税3%引き下げ",
    tau_l=0.17,  # 20% → 17%
    implementation='permanent'
)

print(f"改革案: {income_tax_reform.name}")
print(f"所得税率: {params.tau_l:.1%} → {income_tax_reform.tau_l:.1%}")

# シミュレーション実行
print("\n🔄 シミュレーション実行中...")
income_results = simulator.simulate_reform(
    reform=income_tax_reform,
    periods=20
)
print("✅ シミュレーション完了")

改革案: 所得税3%引き下げ
所得税率: 20.0% → 17.0%

🔄 シミュレーション実行中...
Simulating 所得税3%引き下げ with enhanced engine...
Using tax-adjusted initial guess for large tax change (magnitude: 0.030)
✅ シミュレーション完了


/Users/daisukeyoda/Documents/JapanTaxSimulator/src/tax_simulator.py:279: ResearchWarning: RESEARCH WARNING - simulate_reform: Uses automatic model selection (simple vs complex) with different economic assumptions. May return results from different underlying models without clear indication. Welfare calculations use simplified approximations.
  return self.simulator.simulate_reform(reform, periods, compute_welfare=True)
/Users/daisukeyoda/Documents/JapanTaxSimulator/src/tax_simulator.py:156: ResearchWarning: RESEARCH WARNING - simulate_reform: Uses automatic model selection (simple vs complex) with different economic assumptions. May return results from different underlying models without clear indication. Welfare calculations use simplified approximations.
  results = self.simulation_engine.simulate_reform(reform, periods)
/Users/daisukeyoda/Documents/JapanTaxSimulator/src/tax_simulator.py:160: ResearchWarning: RESEARCH WARNING - analyze_welfare_impact: Welfare calculations use simplif

In [23]:
# 所得税改革の結果分析
if hasattr(income_results, 'steady_state_baseline') and hasattr(income_results, 'steady_state_reform'):
    baseline_ss = income_results.steady_state_baseline
    reform_ss = income_results.steady_state_reform
    
    print("=== 所得税引き下げの長期的影響 ===")
    
    variables = {'Y': 'GDP', 'C': '消費', 'I': '投資', 'L': '労働時間'}
    
    for var, jp_name in variables.items():
        if hasattr(baseline_ss, var) and hasattr(reform_ss, var):
            old_val = getattr(baseline_ss, var)
            new_val = getattr(reform_ss, var)
            if old_val != 0:
                change = (new_val - old_val) / old_val * 100
                direction = "📈" if change > 0 else "📉" if change < 0 else "➡️"
                print(f"{direction} {jp_name}: {change:+.2f}%")
    
    # 福利厚生への影響
    if hasattr(income_results, 'welfare_change'):
        welfare = income_results.welfare_change
        welfare_direction = "📈" if welfare > 0 else "📉" if welfare < 0 else "➡️"
        print(f"\n{welfare_direction} 家計福利: {welfare:+.2f}%")

=== 所得税引き下げの長期的影響 ===
📉 GDP: -6.56%
➡️ 消費: +0.00%
➡️ 投資: +0.00%
📉 労働時間: -1.10%

📈 家計福利: +0.00%


## 政策提言

### 消費税政策
- 消費税引き上げは消費を抑制
- 税収確保には有効だが経済活動には負の影響

### 所得税政策  
- 所得税引き下げは労働供給を刺激
- 経済成長促進効果が期待される
- 税収減少とのトレードオフ

In [24]:
print("✅ 02_tax_policy_fundamentals 完了")
print("\n📋 学習内容:")
print("✅ 日本の税制構造理解")
print("✅ 消費税政策の経済効果分析")
print("✅ 所得税政策の経済効果分析")
print("✅ 政策提言の基礎")
print("\n🚀 次: 03_research_grade_simulation.ipynb")

✅ 02_tax_policy_fundamentals 完了

📋 学習内容:
✅ 日本の税制構造理解
✅ 消費税政策の経済効果分析
✅ 所得税政策の経済効果分析
✅ 政策提言の基礎

🚀 次: 03_research_grade_simulation.ipynb
